<a href="https://colab.research.google.com/github/NARizzardi/machine-learning/blob/main/Trabalho/Trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
import kagglehub
import cv2
import os

path = kagglehub.dataset_download("zlatan599/mushroom1")

mushroomdf_val = pd.read_csv(f"{path}/val.csv") # nao irei usar provavelmente para a analise mas ja deixo ai
mushroomdf_test = pd.read_csv(f"{path}/test.csv") # val kinda related to test
mushroomdf_train = pd.read_csv(f"{path}/train.csv").drop_duplicates(subset=["image_path"])

mushroomdf_val['image_path'] = mushroomdf_val['image_path'].str.replace('/kaggle/working','/kaggle/input/mushroom1')
mushroomdf_train['image_path'] = mushroomdf_train['image_path'].str.replace('/kaggle/working','/kaggle/input/mushroom1')
mushroomdf_test['image_path'] = mushroomdf_test['image_path'].str.replace('/kaggle/working','/kaggle/input/mushroom1')

paths_val = mushroomdf_val.image_path
paths_train = mushroomdf_train.image_path
paths_test = mushroomdf_test.image_path

paths = list(paths_train) + list(paths_test)


# Mais velocidade usando multiprocessamento
from joblib import Parallel, delayed

X_train = []
y_train = []
X_test = []
y_test = []

# These can be found in /content
if os.path.exists("X_train.npy") and os.path.exists("y_train.npy") and os.path.exists("X_test.npy") and os.path.exists("y_test.npy"):
    X_train = np.load("X_train.npy")
    y_train = np.load("y_train.npy", allow_pickle=True)
    X_test = np.load("X_test.npy")
    y_test = np.load("y_test.npy", allow_pickle=True)
    print("Loaded from files")
else:
  print("Loading images...")

  def loadImg(path):
    try:
      img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
      img = cv2.resize(img, (64, 64))
      return img.flatten() / 255.0
    except Exception as e:
      print(f"{e} - Error loading {path}")
      return None

  results = Parallel(n_jobs=-1, prefer="threads")(
      delayed(loadImg)(p)
      for p in paths
  )

  #Removendo cargas falhas
  results = [img for img in results if img is not None]

  X_train = np.array(results[:len(paths_train)])
  y_train = mushroomdf_train.label.to_numpy()

  X_test = np.array(results[len(paths_train):])
  y_test = mushroomdf_test.label.to_numpy()

  # save
  np.save("X_train.npy", X_train)
  np.save("y_train.npy", y_train)
  np.save("X_test.npy", X_test)
  np.save("y_test.npy", y_test)

# ~4 minutos se kaggle nao presente
# ~8-10mins pra tudo ser alocado
# ~18segundos-1minuto  se tudo presente

# Algumas verificacoes para ver se ta tudo certo, pode ser comentado
print(f"Total train paths: {len(paths_train)}, total test paths: {len(paths_test)}")
print(f"Images on train loaded: {len(X_train)}, labels loaded: {len(y_train)}, images on test: {len(X_test)}, labels on test: {len(y_test)}")

print(f"Shape of X_train: {X_train.shape} and of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape} and of y_test: {y_test.shape}")

Using Colab cache for faster access to the 'mushroom1' dataset.
Loaded from files
Total train paths: 72858, total test paths: 15614
Images on train loaded: 72858, labels loaded: 72858, images on test: 15614, labels on test: 15614
Shape of X_train: (72858, 4096) and of y_train: (72858,)
Shape of X_test: (15614, 4096) and of y_test: (15614,)


In [12]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split


def svm(nucleo, subset_size):
  svm_model = SVC(kernel=nucleo, random_state=42, verbose=True)

  X_train_subset, _, y_train_subset, _ = train_test_split(
      X_train, y_train, train_size=subset_size, stratify=y_train, random_state=42
  )

  print(f"Treinando modelo SVM com um subconjunto de {len(X_train_subset)} amostrar...")
  svm_model.fit(X_train_subset, y_train_subset)
  print("Treinamento do modelo SVM completo.")
  return svm_model

svm_model = svm('rbf', 0.2)

KeyboardInterrupt: 

In [11]:

print("Fazendo predições no conjunto de teste...")
y_pred = svm_model.predict(X_test)
print("Predições completas.")

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"\Acurária: {accuracy:.4f}")

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))


Making predictions on the test set...
Predictions complete.

Accuracy: 0.1016

Classification Report:
                             precision    recall  f1-score   support

          Agaricus augustus       0.00      0.00      0.00        51
      Agaricus xanthodermus       0.00      0.00      0.00        46
     Amanita amerirubescens       0.00      0.00      0.00        50
            Amanita augusta       0.00      0.00      0.00        50
        Amanita brunnescens       0.00      0.00      0.00        46
      Amanita calyptroderma       0.00      0.00      0.00        49
            Amanita citrina       0.00      0.00      0.00        44
         Amanita flavoconia       0.00      0.00      0.00        48
           Amanita muscaria       0.08      0.54      0.14       687
         Amanita pantherina       0.00      0.00      0.00        51
          Amanita persicina       0.00      0.00      0.00        39
         Amanita phalloides       0.00      0.00      0.00        48


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
